In [4]:
def read_file():
    f = open('testSet.txt')
    data_set = []
    label = []
    for line in f.readlines():
        string = line.strip().split('\t')
        data_set.append([float(string[0]),float(string[1])])
        label.append(float(string[2]))
    return data_set, label

data_set, label = read_file()
print len(label)

100


In [35]:
import random
def random_select(index, row_num):
    j = index
    // need to caution the scope of variance
    while(j == index):
        j = int(random.uniform(0,row_num))
    return j
         
    
#j = random_select(8, 20)
#print j

5


In [38]:
def adust_alpha(a_j, low, high):
    if a_j > high:
        a_j = high
    if a_j < low:
        a_j = low
    return a_j
print adust_alpha(-1,0, 100)

0


In [98]:
import numpy as np
def smo1(data_set,label,constant, tolence, iteration_num):
    #need to use np.mat(), rather than np.array()
    data_mat = np.mat(data_set)
    label_vec = np.mat(label).T
    row_num, col_num = data_mat.shape
    alpha = np.mat(np.zeros((row_num, 1)))
    iteration_count = 0
    b = 0
    while(iteration_count < iteration_num):
        alpha_flag = 0
        for i in range(row_num):
            predict_i = float(np.multiply(alpha, label_vec).T *(data_mat * data_mat[i,:].T)) + b
            error_i = predict_i - label_vec[i]
            if ((label_vec[i]* error_i < -tolence) and (alpha[i] < constant)) or ((label_vec[i]*error_i > tolence) and (alpha[i] > 0)):
                j = random_select(i, row_num)
                predict_j = float(np.multiply(alpha, label_vec).T * \
                                     (data_mat * data_mat[j,:].T)) + b
                error_j = predict_j - label_vec[j]
                alpha_i_copy = alpha[i].copy()
                alpha_j_copy = alpha[j].copy()
                
                #set low and hign bound
                if(label_vec[i] != label_vec[j]):
                    low_bound = max(0, alpha[j] - alpha[i])
                    high_bound = min(constant, constant + alpha[j] - alpha[i])
                else:
                    low_bound = max(0, alpha[j] + alpha[i] - constant)
                    high_bound = min(constant, alpha[i] + alpha[j])
                if low_bound == high_bound:
                    continue
                #set eta
                eta = 2.0 * data_mat[i,:]*data_mat[j,:].T - data_mat[i,:] * data_mat[i,:].T - data_mat[j,:]*data_mat[j,:].T
                if eta >= 0:
                    continue
                alpha[i] -= label_vec[j] * (error_i - error_j) / eta
                alpha[j] = adust_alpha(alpha[j], low_bound, high_bound)
                
                if (abs(alpha[i] - alpha[j]) < 0.00001):
                    continue
                alpha[i] += label_vec[j] * label_vec[i] * (alpha_j_copy - alpha[j])
                b1 = b- error_i - label_vec[i] * (alpha[i] - alpha_i_copy) * data_mat[i,:]* data_mat[i,:].T - \
                label_vec[j] * (alpha[j] - alpha_j_copy) * data_mat[i,:]*data_mat[j,:].T
                b2 = b - error_j - label_vec[i] * (alpha[i] - alpha_i_copy) *data_mat[i,:]*data_mat[j,:].T - \
                    label_vec[j]*(alpha[j] - alpha_j_copy) * data_mat[j,:] * data_mat[j,:].T
                
                if( alpha[i] > 0) and ( alpha[i] < constant):
                    b = b1
                elif (alpha[j] > 0) and ( alpha[j] < constant):
                    b = b2
                else:
                    b = (b1 + b2) / 2
                alpha_flag += 1
                print 'iteratin : %d', i
        if alpha_flag == 0:
            iteration_count += 1
        else:
            iteration_num = 0
    return alpha, b

alpha, b = smo1(data_set, label,0.6, 0.001, 50)
print alpha
            

iteratin : %d 1
iteratin : %d 2
iteratin : %d 4
iteratin : %d 6
iteratin : %d 10
iteratin : %d 26
iteratin : %d 27
iteratin : %d 29
iteratin : %d 31
iteratin : %d 39
iteratin : %d 42
iteratin : %d 44
iteratin : %d 47
iteratin : %d 49
iteratin : %d 50
iteratin : %d 52
iteratin : %d 55
iteratin : %d 97
(100L, 1L)


In [94]:
import numpy as np
label_voc = np.mat(label).T
alpha = np.zeros((100,1))
x = np.multiply(alpha, label_voc).T
data_mat = np.mat(data_set)
y = data_mat * (data_mat[2,:].T)
print data_mat[2,:].T.shape


(2L, 1L)
